<a href="https://colab.research.google.com/github/cedro3/first-order-model/blob/master/demo2_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Order Motion Model DEMO

# コード全体を git clone

In [ ]:
!git clone https://github.com/cedro3/first-order-model.git

In [ ]:
cd first-order-model

# サンプルデータ (sample.zip)　をダウンロード

In [ ]:
# Google drive から サンプルデータ (sample.zip) をダウンロードする
import requests

def download_file_from_google_drive(id, destination):

       # ダウンロード画面のURL
       URL = "https://drive.google.com/uc?id=1qHX6hFJlWVdzsQZASNLsoQsDlTccyQce&export=download" 

       session = requests.Session()

       response = session.get(URL, params = { 'id' : id }, stream = True)
       token = get_confirm_token(response)

       if token:
           params = { 'id' : id, 'confirm' : token }
           response = session.get(URL, params = params, stream = True)

       save_response_content(response, destination)    

def get_confirm_token(response):
       for key, value in response.cookies.items():
           if key.startswith('download_warning'):
               return value

       return None

def save_response_content(response, destination):
       CHUNK_SIZE = 32768

       with open(destination, "wb") as f:
           for chunk in response.iter_content(CHUNK_SIZE):
               if chunk: # filter out keep-alive new chunks
                   f.write(chunk)

if __name__ == "__main__":
       file_id = 'TAKE ID FROM SHAREABLE LINK' 
       destination = './sample.zip'  # 保存先パスの指定
       download_file_from_google_drive(file_id, destination)

In [ ]:
# sample.zip ファイルを解凍する
import zipfile
zipf = zipfile.ZipFile('sample.zip')
zipf.extractall()
zipf.close()

# コード本体

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=33, repeat_delay=1000)
    plt.close()
    return ani

# 「モナリザ」の静止画を「トランプ大統領」の動画のように動かす

In [ ]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('./sample/05.png')
driving_video = imageio.mimread('./sample/04.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./sample/vox-cpk.pth.tar')

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

#オリジナル動画を使う

In [ ]:
# ABCニュースの動画からキャスターの顔部分だけを切り出し
!ffmpeg -i ./sample/news.mp4 -ss 00:00:00.00 -t 00:00:10 -filter:v "crop=200:200:550:80" -async 1 result.mp4

In [ ]:
# result.mp4 を動画にして、5枚の静止画から生成動画のフレームを作成する

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

s0 = resize(imageio.imread('./sample/pic0.png'), (256, 256))[..., :3]
s1 = resize(imageio.imread('./sample/pic1.png'), (256, 256))[..., :3]
s2 = resize(imageio.imread('./sample/pic2.png'), (256, 256))[..., :3]
s3 = resize(imageio.imread('./sample/pic3.png'), (256, 256))[..., :3]
s4 = resize(imageio.imread('./sample/pic4.png'), (256, 256))[..., :3]
source_image = [s0, s1, s2, s3, s4]


driving_video = imageio.mimread('./result.mp4')
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./sample/vox-cpk.pth.tar')

result = []
for i in range(5):
    predictions = make_animation(source_image[i], driving_video, generator, kp_detector, relative=True)
    result.append(predictions)

In [ ]:
# 5枚の静止画の生成動画と動画を連結します。

from tqdm import trange
fig = plt.figure(figsize=(12, 8))
ims =[]
for i in trange(len(result[0])):
    x = np.concatenate([result[0][i], result[1][i], result[2][i]],1)
    y = np.concatenate([result[3][i], driving_video[i], result[4][i]],1)
    z = np.concatenate([x, y])
    im = plt.imshow(z, animated=True)
    plt.axis('off')
    ims.append([im])

print('making animeation...')
ani = animation.ArtistAnimation(fig, ims, interval=33, repeat_delay=1000)
plt.close()
HTML(ani.to_html5_video())